# Tarea 05: Optimización de hiperparámetros

En la carpeta de data https://github.com/scidatmath2020/ML_Py_23/tree/main/data   encontrarás la tabla datos_películas.csv. Aplica todos los modelos de clasificación muticlase que hemos visto, con score F1, para encontrar el mejor clasificador.

¿Cuál es el mejor modelo para clasificar con esta tabla?

In [2]:
# Importamos las librerías estandar para manipular nuestro dataframe
import os
import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import * 
from plotnine import *
import time

In [3]:
######### Importar datos ##########

# Importamos los datos de las películas desde Github
mi_data = pd.read_csv('https://raw.githubusercontent.com/scidatmath2020/ML_Py_23/main/data/datos_peliculas.csv')
# Visualizamos las primeras 5 filas
mi_data.head()

,pelicula,año,ratings,genero,ventas,presupuesto,secuela,vistas_youtube,positivos_youtube,negativos_youtube,comentarios,seguidores_agregados
0,13 Sins,2014,6.3,8,9130,4000000.0,1,3280543,4632,425,636,1120000.0
1,22 Jump Street,2014,7.1,1,192000000,50000000.0,2,583289,3465,61,186,12350000.0
2,3 Days to Kill,2014,6.2,1,30700000,28000000.0,1,304861,328,34,47,483000.0
3,300: Rise of an Empire,2014,6.3,1,106000000,110000000.0,2,452917,2429,132,590,568000.0
4,A Haunted House 2,2014,4.7,8,17300000,3500000.0,2,3145573,12163,610,1082,1923800.0


In [4]:
# Nuestra columna objetivo es 'genero', el cual nos describe el genero de la película. Al entrenar los
# modelos queremos predecir de qué genero es una película basado en las columnas numéricas.
# Checamos que clases tenemos en 'genero'
mi_data.genero.unique()

array([ 8,  1,  3, 10, 15, 12,  9,  2,  7,  6,  4], dtype=int64)

In [5]:
# Ahora chequemos si la columna 'genero' tiene clases balanceadas o desbalanceadas
mi_data >> group_by(_.genero) >> summarize(total = n(_))

,genero,total
0,1,65
1,2,12
2,3,46
3,4,1
4,6,3
5,7,2
6,8,54
7,9,13
8,10,12
9,12,13


Como vemos hay clases desbalanceadas en este dataset. Por ejemplo, los géneros '4', '6' o '7'. Será importante tomar esto en cuenta en lo
subsiguiente.

In [6]:
######### Modelos de clasificación multiclase ##########

# Importamos las librerías de sklearn que nos permiten clasificar datos multiclase
# Estos son: K-vecinos, árboles de decisión y máquinas de soporte vectorial
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

In [7]:
# Definimos como X al dataframe con los valores independientes. Para esto vamos a considerar todas los valores numéricos
# Como usaremos modelos que se basan en distancias, vamos a omitir las columnas 'pelicula', 'año' y 'secuela'
# Estos son todas las columnas excepto 'Species'.
X = mi_data >> select(-_.pelicula, -_.año, -_.secuela, -_.genero)

# Definimos como y al dataframe con los valores objetivo, los cuales se encuentran en la columna 'Species'
y =  mi_data >> select(_.genero)

In [7]:
def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring="f1_micro", n_jobs=-1, cv=5)
    return resultados_estimador

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_cols:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].max()
    return resultados_df >> arrange(-_.test_score,_.fit_time)

resultados = {}

In [8]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [10]:
model_tree = tree.DecisionTreeClassifier()
parametros_tree = {
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": [2, 3, 4, 5, 6, 7, 8]
}

tree_grid = GridSearchCV(estimator = model_tree, 
                    param_grid = parametros_tree,
                    scoring = "f1_weighted", n_jobs = -1)

start_time = time.time()
tree_grid.fit(X, y.values.ravel())
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time} seconds")

print(tree_grid.best_score_)
print(tree_grid.best_estimator_.get_params())

resultados["tree_gridsearch"] = evaluar_modelo(tree_grid.best_estimator_,
                                             X,
                                             y.values.ravel())

ver_resultados()

c:\Users\wikki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.


Elapsed time: 4.57179069519043 seconds
0.34464847689945516
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}


c:\Users\wikki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.


,fit_time,score_time,test_score,fit_time_idx,score_time_idx,test_score_idx
tree_gridsearch,0.007058,0.002569,0.393525,1.0,1.0,1.0


In [11]:
model_knn = KNeighborsClassifier()
parametros_knn = {
    "n_neighbors": [1, 10, 20, 30, 40, 50],
    "p": [1, 2, 3],
    "weights": ["uniform", "distance"]
}


knn_grid = GridSearchCV(estimator = model_knn, 
                    param_grid = parametros_knn,
                    scoring = "f1_weighted", n_jobs = -1)

start_time = time.time()
knn_grid.fit(X, y.values.ravel())
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time} seconds")

print(knn_grid.best_score_)
print(knn_grid.best_estimator_.get_params())

resultados["knn_gridsearch"] = evaluar_modelo(knn_grid.best_estimator_,
                                             X,
                                             y.values.ravel())

ver_resultados()

c:\Users\wikki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.


Elapsed time: 0.48362231254577637 seconds
0.299490184708353
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 20, 'p': 1, 'weights': 'distance'}


c:\Users\wikki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.


,fit_time,score_time,test_score,fit_time_idx,score_time_idx,test_score_idx
tree_gridsearch,0.007058,0.002569,0.393525,1.000000,0.432339,1.000000
knn_gridsearch,0.003615,0.005943,0.354764,0.512155,1.000000,0.901504


In [12]:
model_svm = SVC()

parametros_svm = {
    "degree": [1, 2, 3, 4],
    "gamma": [0.1, 0.5, 1.0, 10.0],
    "kernel": ["poly", "rbf"]
}

svm_grid = GridSearchCV(estimator=model_svm, 
                        param_grid=parametros_svm,
                        scoring="f1_weighted", n_jobs=-1)

start_time = time.time()
svm_grid.fit(X, y.values.ravel())
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time} seconds")

print(svm_grid.best_score_)
print(svm_grid.best_estimator_.get_params())

resultados["svm_gridsearch"] = evaluar_modelo(svm_grid.best_estimator_,
                                             X,
                                             y.values.ravel())

ver_resultados()


c:\Users\wikki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
